In [1]:
import pandas as pd
import numpy as np
from mealpy.optimizer import Optimizer

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import time            
import numpy as np

from mealpy.utils import io
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# Assuming you have a dataset 'X' and corresponding labels 'y'

datasetPoly = pd.read_csv('labelled-balanced-data11.csv')
X, y = datasetPoly.iloc[:, :-1], datasetPoly.iloc[:, -1]

# Determine the number of features
num_features = X.shape[1]




In [2]:
class NomadZOA(Optimizer):
    def __init__(self, epoch=10000, pop_size=100, **kwargs):
        super().__init__(**kwargs)
        self.epoch = self.validator.check_int("epoch", epoch, [1, 100000])
        self.pop_size = self.validator.check_int("pop_size", pop_size, [10, 10000])
        self.set_parameters(["epoch", "pop_size"])
        self.sort_flag = False
        
    def evolve(self, epoch):
        
        # PHASE1: Foraging Behaviour
        pop_new = []
        for idx in range(0, self.pop_size):
            r1 = np.round(1 + np.random.rand())
            pos_new = self.pop[idx][self.ID_POS] + np.random.rand(self.problem.n_dims) * (self.g_best[self.ID_POS] - r1 * self.pop[idx][self.ID_POS])   # Eq. 3
            pos_new = self.amend_position(pos_new, self.problem.lb, self.problem.ub)
            pop_new.append([pos_new, None])
            if self.mode not in self.AVAILABLE_MODES:
                target = self.get_target_wrapper(pos_new)
                self.pop[idx] = self.get_better_solution([pos_new, target], self.pop[idx])
        if self.mode in self.AVAILABLE_MODES:
            pop_new = self.update_target_wrapper_population(pop_new)
            self.pop = self.greedy_selection_population(self.pop, pop_new)
            
        # PHASE2: Defense strategies against predators
        kk = np.random.permutation(self.pop_size)[0]
        pop_new = []
        while True:
            if np.random.rand() < 0.5:
                # S1: the lion attacks the zebra and thus the zebra chooses an escape strategy
                r2 = 0.1
                pos_new = self.pop[idx][self.ID_POS] + r2 * (2 + np.random.rand(self.problem.n_dims) - 1) * (1 - (epoch+1)/self.epoch) * self.pop[idx][self.ID_POS]
            else:
                # S2: other predators attack the zebra and the zebra will choose the offensive strategy
                r2 = np.random.randint(1, 3)
                pos_new = self.pop[idx][self.ID_POS] + np.random.rand(self.problem.n_dims) * (self.pop[kk][self.ID_POS] - r2 * self.pop[idx][self.ID_POS])

            pos_new = self.amend_position(pos_new, self.problem.lb, self.problem.ub)

            # Check if the newly generated pos_new is different from the current position
            if not np.allclose(pos_new, self.pop[idx][self.ID_POS]):
                break  # Exit the loop if a different position is found


In [4]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE
from sklearn.metrics import accuracy_score

# Assuming X and y are defined elsewhere in your code

# Define a fitness function that uses Recursive Feature Elimination (RFE) with Random Forest
def fitness_function(solution):
    selected_features = [i for i, selected in enumerate(solution) if selected]
    if len(selected_features) < 2:
        return 0.0  # No features selected, return fitness value of 0
    
    X_subset = X.iloc[:, selected_features]  # Select the corresponding columns from the dataset

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X_subset, y, test_size=0.25, random_state=42)

    # Create a Random Forest classifier
    rfc = RandomForestClassifier()

    # Use Recursive Feature Elimination (RFE)
    rfe = RFE(estimator=rfc, n_features_to_select=len(selected_features))
    X_train_rfe = rfe.fit_transform(X_train, y_train)
    X_test_rfe = rfe.transform(X_test)

    # Train the classifier on the subset of features selected by RFE
    rfc.fit(X_train_rfe, y_train)

    # Predict the labels for the test set
    y_pred = rfc.predict(X_test_rfe)

    # Calculate the accuracy score as the fitness value
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

# Assuming num_features is defined elsewhere in your code
num_features = X.shape[1]

problemRfeOZ = {
    "fit_func": fitness_function,
    "lb": [0] * num_features,  # Lower bound for each feature (0 means not selected)
    "ub": [1] * num_features,  # Upper bound for each feature (1 means selected)
    "minmax": "max",  # Maximizing the fitness function
    "save_population": True
}

# Number of optimization runs
num_runs = 10

# Store results
all_results_rfe_nzoa = []

for run in range(num_runs):
    # Create the PoliticalZOA optimization model
    modelrfnomad = NomadZOA(epoch=100, pop_size=100)

    # Measure the total execution time
    start_time = time.time()

    # Solve the optimization problem
    best_position, best_fitness = modelrfnomad.solve(problemRfeOZ)

    # Calculate and print the total execution time
    end_time = time.time()
    total_time = end_time - start_time
    print(f"Run {run + 1}: Total execution time: {total_time} seconds")

    # Create a dictionary to map feature names to their corresponding position values
    feature_position_dict = {feature_name: position for feature_name, position in zip(X.columns, best_position)}

    # Store results
    all_results_rfe_nzoa.append({"position_values": feature_position_dict, "fitness": best_fitness})



2023/12/30 09:57:40 PM, INFO, __main__.NomadZOA: Solving single objective optimization problem.
2023/12/31 10:03:21 AM, INFO, __main__.NomadZOA: >Problem: P, Epoch: 1, Current best: 0.8627049180327869, Global best: 0.8627049180327869, Runtime: 145.86203 seconds
2023/12/31 10:06:08 AM, INFO, __main__.NomadZOA: >Problem: P, Epoch: 2, Current best: 0.8627049180327869, Global best: 0.8627049180327869, Runtime: 166.24464 seconds
2023/12/31 10:08:31 AM, INFO, __main__.NomadZOA: >Problem: P, Epoch: 3, Current best: 0.8627049180327869, Global best: 0.8627049180327869, Runtime: 143.70929 seconds
2023/12/31 10:10:57 AM, INFO, __main__.NomadZOA: >Problem: P, Epoch: 4, Current best: 0.8627049180327869, Global best: 0.8627049180327869, Runtime: 145.31172 seconds
2023/12/31 10:13:21 AM, INFO, __main__.NomadZOA: >Problem: P, Epoch: 5, Current best: 0.8627049180327869, Global best: 0.8627049180327869, Runtime: 144.83033 seconds
2023/12/31 10:15:46 AM, INFO, __main__.NomadZOA: >Problem: P, Epoch: 6, Cu

2023/12/31 12:05:59 PM, INFO, __main__.NomadZOA: >Problem: P, Epoch: 50, Current best: 0.8637295081967213, Global best: 0.8637295081967213, Runtime: 144.50429 seconds
2023/12/31 12:08:25 PM, INFO, __main__.NomadZOA: >Problem: P, Epoch: 51, Current best: 0.8637295081967213, Global best: 0.8637295081967213, Runtime: 146.41002 seconds
2023/12/31 12:10:51 PM, INFO, __main__.NomadZOA: >Problem: P, Epoch: 52, Current best: 0.8637295081967213, Global best: 0.8637295081967213, Runtime: 145.55571 seconds
2023/12/31 12:13:16 PM, INFO, __main__.NomadZOA: >Problem: P, Epoch: 53, Current best: 0.8637295081967213, Global best: 0.8637295081967213, Runtime: 145.01523 seconds
2023/12/31 12:15:35 PM, INFO, __main__.NomadZOA: >Problem: P, Epoch: 54, Current best: 0.8637295081967213, Global best: 0.8637295081967213, Runtime: 139.52779 seconds
2023/12/31 12:17:58 PM, INFO, __main__.NomadZOA: >Problem: P, Epoch: 55, Current best: 0.8637295081967213, Global best: 0.8637295081967213, Runtime: 142.79858 second

2023/12/31 02:07:09 PM, INFO, __main__.NomadZOA: >Problem: P, Epoch: 100, Current best: 0.8637295081967213, Global best: 0.8637295081967213, Runtime: 146.02298 seconds


Run 1: Total execution time: 58170.449662446976 seconds


2023/12/31 02:07:11 PM, INFO, __main__.NomadZOA: Solving single objective optimization problem.
2023/12/31 02:12:06 PM, INFO, __main__.NomadZOA: >Problem: P, Epoch: 1, Current best: 0.8611680327868853, Global best: 0.8611680327868853, Runtime: 143.58554 seconds
2023/12/31 02:14:30 PM, INFO, __main__.NomadZOA: >Problem: P, Epoch: 2, Current best: 0.8657786885245902, Global best: 0.8657786885245902, Runtime: 144.01479 seconds
2023/12/31 02:16:54 PM, INFO, __main__.NomadZOA: >Problem: P, Epoch: 3, Current best: 0.8657786885245902, Global best: 0.8657786885245902, Runtime: 144.36972 seconds
2023/12/31 02:19:19 PM, INFO, __main__.NomadZOA: >Problem: P, Epoch: 4, Current best: 0.8657786885245902, Global best: 0.8657786885245902, Runtime: 144.30669 seconds
2023/12/31 02:21:44 PM, INFO, __main__.NomadZOA: >Problem: P, Epoch: 5, Current best: 0.8657786885245902, Global best: 0.8657786885245902, Runtime: 145.35690 seconds
2023/12/31 02:24:07 PM, INFO, __main__.NomadZOA: >Problem: P, Epoch: 6, Cu

...

Run 9: Total execution time: 15697.923300027847 seconds


2024/01/03 01:28:17 AM, INFO, __main__.NomadZOA: Solving single objective optimization problem.
2024/01/03 01:33:09 AM, INFO, __main__.NomadZOA: >Problem: P, Epoch: 1, Current best: 0.8621926229508197, Global best: 0.8621926229508197, Runtime: 141.21746 seconds
2024/01/03 01:35:28 AM, INFO, __main__.NomadZOA: >Problem: P, Epoch: 2, Current best: 0.8621926229508197, Global best: 0.8621926229508197, Runtime: 138.67791 seconds
2024/01/03 01:37:49 AM, INFO, __main__.NomadZOA: >Problem: P, Epoch: 3, Current best: 0.8621926229508197, Global best: 0.8621926229508197, Runtime: 140.76252 seconds
2024/01/03 01:40:13 AM, INFO, __main__.NomadZOA: >Problem: P, Epoch: 4, Current best: 0.8621926229508197, Global best: 0.8621926229508197, Runtime: 144.42652 seconds
2024/01/03 01:42:34 AM, INFO, __main__.NomadZOA: >Problem: P, Epoch: 5, Current best: 0.8621926229508197, Global best: 0.8621926229508197, Runtime: 141.10571 seconds
2024/01/03 01:44:57 AM, INFO, __main__.NomadZOA: >Problem: P, Epoch: 6, Cu

2024/01/03 03:26:01 AM, INFO, __main__.NomadZOA: >Problem: P, Epoch: 50, Current best: 0.8632172131147541, Global best: 0.8632172131147541, Runtime: 140.88230 seconds
2024/01/03 03:28:25 AM, INFO, __main__.NomadZOA: >Problem: P, Epoch: 51, Current best: 0.8632172131147541, Global best: 0.8632172131147541, Runtime: 143.48176 seconds
2024/01/03 03:30:49 AM, INFO, __main__.NomadZOA: >Problem: P, Epoch: 52, Current best: 0.8632172131147541, Global best: 0.8632172131147541, Runtime: 144.69320 seconds
2024/01/03 03:33:12 AM, INFO, __main__.NomadZOA: >Problem: P, Epoch: 53, Current best: 0.8632172131147541, Global best: 0.8632172131147541, Runtime: 142.76733 seconds
2024/01/03 03:35:35 AM, INFO, __main__.NomadZOA: >Problem: P, Epoch: 54, Current best: 0.8632172131147541, Global best: 0.8632172131147541, Runtime: 143.25373 seconds
2024/01/03 03:37:59 AM, INFO, __main__.NomadZOA: >Problem: P, Epoch: 55, Current best: 0.8632172131147541, Global best: 0.8632172131147541, Runtime: 143.75799 second

2024/01/03 05:25:02 AM, INFO, __main__.NomadZOA: >Problem: P, Epoch: 100, Current best: 0.8637295081967213, Global best: 0.8637295081967213, Runtime: 137.09610 seconds


Run 10: Total execution time: 14206.479204416275 seconds


In [5]:
# Print the feature names and corresponding position values for each run in sorted order
for run, result in enumerate(all_results_rfe_nzoa, start=1):
    sorted_features = sorted(result["position_values"].items(), key=lambda x: x[1])
    print(f"\nRun {run}: Fitness (Accuracy): {result['fitness']}")
    for feature_name, position_value in sorted_features:
        print(f"Feature: {feature_name}, Position Value: {position_value}")


Run 1: Fitness (Accuracy): 0.8637295081967213
Feature: Diet, Position Value: 0.062283807654778915
Feature: Waist_Size, Position Value: 0.21909133088875923
Feature: Family_History, Position Value: 0.24622286332545915
Feature: BMI, Position Value: 0.26658792418233407
Feature: Gender, Position Value: 0.2757270761993993
Feature: Physical_Activity, Position Value: 0.2775481495075881
Feature: Age, Position Value: 0.35961191136459114
Feature: BP, Position Value: 0.4295298504980752
Feature: Region, Position Value: 0.4830855934545446
Feature: Smoking, Position Value: 0.6281436124158137

Run 2: Fitness (Accuracy): 0.8657786885245902
Feature: Diet, Position Value: 0.16872068821495562
Feature: BP, Position Value: 0.21917845615032605
Feature: Age, Position Value: 0.23935772790699514
Feature: Physical_Activity, Position Value: 0.3257635239884868
Feature: Family_History, Position Value: 0.38134955128115433
Feature: Waist_Size, Position Value: 0.46452876289883444
Feature: Smoking, Position Value: 0.5